# [Kaggle Competition](https://www.kaggle.com/c/ga-customer-revenue-prediction/data): Google Analytics Customer Revenue Prediction
Predict how much GStore customers will spend  
**innogyDSGuild**

In [ ]:
import os, time, random
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
random.seed(123)
#random.seed(time.time())

## Read & clean datasets

### Read function
Credit: https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

In [ ]:
def load_df(csv_path='data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

### Clean function
Credit: https://www.kaggle.com/ogakulov/feature-engineering-step-by-step  

In [ ]:
def clean_df(raw_df, copy=False):
    
    if copy:
        raw_df = raw_df.copy()
    
    # Drop columns with just one value or all unknown
    cols_to_drop = [col for col in raw_df.columns if raw_df[col].nunique() == 1]
    raw_df.drop(columns = cols_to_drop, inplace=True)

    # Drop campaign colum as it only has one non-null value
    raw_df.drop(['trafficSource.campaign'], axis=1, inplace=True)
    
    # Rename long column names to be more concise
    raw_df.rename(columns={col_name: col_name.split('.')[-1] for col_name in raw_df.columns}, inplace = True)
    
    # Fill transactionRevenue with zeros and convert its type to numeric for train data
    try:
        raw_df['transactionRevenue'].fillna(0, inplace=True)
        raw_df['transactionRevenue'] = pd.to_numeric(raw_df['transactionRevenue'])
    except KeyError:
        pass
    
    # convert timestamp to DateTime
    raw_df["visitStartTime_POSIX"] = raw_df["visitStartTime"].copy() # backup
    raw_df["visitStartTime"] = raw_df.visitStartTime.apply(pd.Timestamp, unit='s')
    
    return raw_df

### Load & clean data

#### Training data

In [ ]:
%%time
raw_df_train = load_df("data/train.csv", nrows=100000)
df_train = clean_df(raw_df_train)

#### Testing data

or alternatively use some of the training data as testing data, to make verification easier without uploading to kaggle

In [ ]:
visitorIds = list(set(df_train.fullVisitorId))
visitorIds.sort()
random.shuffle(visitorIds)
test_fraction = 0.3
testIds  = visitorIds[:int(test_fraction * len(visitorIds))]
trainIds = visitorIds[int(test_fraction * len(visitorIds)):]

df_train_total = df_train.copy()
df_test  = df_train[df_train.fullVisitorId.isin(testIds) ].copy()
df_train = df_train[df_train.fullVisitorId.isin(trainIds)].copy()

# compute total revenue for test data
logRevenue = df_test[["transactionRevenue","fullVisitorId"]].groupby("fullVisitorId").sum()
logRevenue["LogRevenue"] = np.log(logRevenue["transactionRevenue"] + 1)
logRevenue.drop(columns="transactionRevenue", inplace=True)

# backup true revenue
df_test["trueTransactionRevenue"] = df_test.transactionRevenue.copy()
df_test["transactionRevenue"] = 0

## Exploratory data analysis (EDA)
Check also https://www.kaggle.com/captcalculator/a-very-extensive-gstore-exploratory-analysis

### Check columns of test and train

In [ ]:
print("%-25s | %-10s | %-10s" % ("Column", "Train", "Test"))
print(51*"-")
for s in set(df_train.columns).union(set(df_test.columns)):
    print("%-25s | %10s | %10s" % (s, s in df_train.columns, s in df_test.columns))

## Feature engineering
Some ideas:
* Convert time to users local time
* Use geographic information to add external data sources, like economic wealth of the country ([May not be allowed](https://www.kaggle.com/c/ga-customer-revenue-prediction/discussion/66425 "On The Use of External Data"))
* Add past total revenue, past average revenue per visit, time since last revenue

### Find timezone offset
Trivially by finding the hour of the day with minimum activity in each region and computing the offset to United Kingdom.
Could be done much more accurate with the region/country/metro/city information with additional data sources.

In [ ]:
region_min = {}
for region in set(df_train_total.region):
    mask = df_train_total.region == region
    hist, bin_edges = np.histogram(df_train_total.visitStartTime[mask].dt.hour, bins=24, range=(0,24))
    region_min[region] = bin_edges[hist.argmin()]

In [ ]:
mask = df_train.country == "United States"
plt.hist(df_train.visitStartTime[mask].dt.hour, bins=24, range=(0,24))
plt.title("Visits by hour of day for US")

## Predict *transactionRevenue*
For the moment, fill with random values

or zeros

In [ ]:
df_test["transactionRevenue"] = 0

### Try [SGDRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) from scikit learn

In [ ]:
from sklearn import linear_model

Train classifier with hour of day of the transaction

In [ ]:
clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(df_train.visitStartTime.dt.hour.values.reshape(-1, 1), df_train.transactionRevenue)

In [ ]:
df_test["transactionRevenue"] = clf.predict(df_test.visitStartTime.dt.hour.values.reshape(-1, 1))

## Accumulate revenue per fullVisitorID and write output for submission

In [ ]:
prediction = df_test[["transactionRevenue","fullVisitorId"]].groupby("fullVisitorId").sum()
prediction["PredictedLogRevenue"] = np.log(prediction["transactionRevenue"] + 1)
prediction.drop(columns="transactionRevenue", inplace=True)
prediction.head()
prediction.to_csv("prediction.csv")

### Compute RMSD to true data if possible

In [ ]:
RMSD = ((prediction.PredictedLogRevenue - logRevenue.LogRevenue)**2).mean()**0.5
print("RMSD = %.4f" % RMSD)

Predicting the transaction revenue to be zero always results in an RMSD of less than 2.5

## Check where we are on the [public leaderboard from kaggle](https://www.kaggle.com/c/ga-customer-revenue-prediction/leaderboard)
Download up to date data first

In [ ]:
ourScore = RMSD

In [ ]:
leaderboard = pd.read_csv("data/publicleaderboarddata.zip")
best = leaderboard[["TeamId", "Score"]].groupby(["TeamId"]).min()
best.sort_values(by="Score", ascending=True, inplace=True)

In [ ]:
ourRank = (best.Score <= ourScore).sum()

In [ ]:
plt.plot(best.Score.values, ".")
plt.plot(2*[ourRank], plt.ylim(), "r")
plt.plot(plt.xlim(), 2*[ourScore], "r")
plt.ylim(1, 5)
plt.xlabel("Rank")
plt.ylabel("Score")

print("Our Rank : %d/%d (best %2.1f%%)" % (ourRank, best.shape[0], 100*ourRank/best.shape[0]))
print("Our Score: %.4f" % ourScore)